In [20]:
import pandas as pd
import numpy as np

df = pd.read_csv("hospital_clean_base.csv")

df.shape

(438, 17)

In [22]:
df.head()

,facility_id,facility_name,county,hospital_type,teaching_rural,bed_size_category,licensed_beds,staffed_beds,occupied_beds,patient_days,discharges,operating_expense,net_patient_revenue,net_income,flag_zero_patient_days,flag_zero_beds,flag_negative_income
0,106580996,ADVENTIST HEALTH AND RIDEOUT,Yuba,Comparable,Non-Teaching,200 - 299,221,151,147.89,54126,11372,525727747,482851282,-48743941,False,False,True
1,106150788,ADVENTIST HEALTH BAKERSFIELD,Kern,Comparable,Non-Teaching,300 - 499,301,198,198.63,72699,17945,704834525,698464738,-5197092,False,False,True
2,106171049,ADVENTIST HEALTH CLEARLAKE,Lake,Comparable,Rural,1 - 49,25,18,15.66,5731,1497,209729803,183332113,-17011138,False,False,True
3,106150706,ADVENTIST HEALTH DELANO,Kern,Comparable,Non-Teaching,150 - 199,156,69,64.20,23496,2614,111157919,100703448,220351,False,False,False
4,106190323,ADVENTIST HEALTH GLENDALE,Los Angeles,Comparable,Non-Teaching,500 +,515,313,310.87,113777,19036,615072638,563092249,-44826471,False,False,True


In [23]:
# Define rows eligible for metric calculations
df_metrics_base = df[
    (df["patient_days"].notna()) &
    (df["patient_days"] > 0) &
    (df["staffed_beds"].notna()) &
    (df["staffed_beds"] > 0) &
    (df["net_patient_revenue"].notna()) &
    (df["net_patient_revenue"] != 0)
].copy()

print("Total rows:", len(df))
print("Rows eligible for metric math:", len(df_metrics_base))
print("Rows excluded from metric math:", len(df) - len(df_metrics_base))

# Sanity check excluded rows
df.loc[df.index.difference(df_metrics_base.index), [
    "facility_name",
    "patient_days",
    "staffed_beds",
    "net_patient_revenue",
    "flag_zero_patient_days",
    "flag_zero_beds"
]].head(5)


Total rows: 438
Rows eligible for metric math: 435
Rows excluded from metric math: 3


,facility_name,patient_days,staffed_beds,net_patient_revenue,flag_zero_patient_days,flag_zero_beds
73,COALINGA STATE HOSPITAL,490037,1450,0,False,False
188,KAISER FOUNDATION NORTHERN REGION,0,0,14087743886,True,True
189,KAISER FOUNDATION SOUTHERN REGION,0,0,10894487347,True,True


In [24]:
# Create decision level metrics
df_metrics_base["utilization_rate"] = (
    df_metrics_base["occupied_beds"] / df_metrics_base["staffed_beds"]
)

df_metrics_base["cost_per_patient_day"] = (
    df_metrics_base["operating_expense"] / df_metrics_base["patient_days"]
)

df_metrics_base["revenue_per_patient_day"] = (
    df_metrics_base["net_patient_revenue"] / df_metrics_base["patient_days"]
)

df_metrics_base["margin_per_patient_day"] = (
    df_metrics_base["net_income"] / df_metrics_base["patient_days"]
)

df_metrics_base["operating_margin"] = (
    df_metrics_base["net_income"] / df_metrics_base["net_patient_revenue"]
)

metric_cols = [
    "utilization_rate",
    "cost_per_patient_day",
    "revenue_per_patient_day",
    "margin_per_patient_day",
    "operating_margin",
]

# Sanity check for infinities or nonsense
print("Any infinity values:", np.isinf(df_metrics_base[metric_cols]).any().any())

df_metrics_base[metric_cols].describe(percentiles=[0.25, 0.5, 0.75]).T


Any infinity values: False


,count,mean,std,min,25%,50%,75%,max
utilization_rate,435.00,0.94,0.11,0.09,0.91,0.96,0.99,1.46
cost_per_patient_day,435.00,"9,235.86","10,665.08",328.24,"2,561.20","7,001.96","11,866.04","86,032.09"
revenue_per_patient_day,435.00,"8,860.93","10,109.10",352.07,"2,432.04","6,548.92","11,399.51","85,271.00"
margin_per_patient_day,435.00,398.85,"2,681.67","-25,038.87",-104.27,184.41,862.84,"18,291.19"
operating_margin,435.00,0.03,0.31,-2.41,-0.02,0.04,0.12,2.81


utilization_rate mean is 0.94 and max is 1.46.

That is not plausible for hospital utilization if it is truly occupied beds divided by staffed beds.

Utilization above 1 means, on average, more beds are occupied than staffed.
That can happen occasionally due to overflow, but not at scale and not up to 1.46.

In [25]:
# Inspect rows with utilization > 1
df_metrics_base.loc[
    df_metrics_base["utilization_rate"] > 1,
    [
        "facility_name",
        "staffed_beds",
        "occupied_beds",
        "patient_days",
        "utilization_rate"
    ]
].sort_values("utilization_rate", ascending=False).head(10)


,facility_name,staffed_beds,occupied_beds,patient_days,utilization_rate
379,STANISLAUS SURGICAL HOSPITAL,1,1.46,376,1.46
434,WEST ANAHEIM MEDICAL CENTER,110,132.12,48356,1.20
233,"MARTIN LUTHER KING, JR. COMMUNITY HOSPITAL",123,141.46,51775,1.15
275,PALMDALE REGIONAL MEDICAL CENTER,131,144.67,52949,1.10
120,FREMONT HOSPITAL,120,124.67,45629,1.04
354,SHRINERS HOSPITAL FOR CHILDREN - NORTHERN CALI...,21,21.79,7975,1.04
353,SHERMAN OAKS HOSPITAL,82,84.99,31108,1.04
28,AMERICAN RECOVERY CENTER,76,78.22,28628,1.03
131,GOOD SAMARITAN HOSPITAL - BAKERSFIELD,32,32.82,12011,1.03
27,ALVARADO PARKWAY INSTITUTE BHS,61,62.19,22763,1.02


What is happening, occupied_beds here is average daily census, not a hard capped bed count.

Establish what normal looks like across the full hospital system before comparing subgroups.
This prevents overreacting to outliers and anchors later comparisons in system context.

In [26]:
# System level context using mean and median
system_summary = (
    df_metrics_base[metric_cols]
    .agg(["mean", "median"])
    .T
)

system_summary


,mean,median
utilization_rate,0.94,0.96
cost_per_patient_day,"9,235.86","7,001.96"
revenue_per_patient_day,"8,860.93","6,548.92"
margin_per_patient_day,398.85,184.41
operating_margin,0.03,0.04


The median hospital operates near full capacity, with utilization close to one.
Median cost per patient day exceeds median revenue per patient day, which explains why many hospitals operate near breakeven or at a loss.
System level operating margins are thin, indicating small efficiency changes can materially impact profitability.

Identify which operational metrics meaningfully separate loss making hospitals from profitable ones.
If no differences exist, targeted intervention is not possible.

In [27]:
# Compare metrics for loss making vs profitable hospitals
group_summary = (
    df_metrics_base
    .groupby("flag_negative_income")[metric_cols]
    .mean()
    .rename(index={False: "Profitable", True: "Loss making"})
)

group_summary


,utilization_rate,cost_per_patient_day,revenue_per_patient_day,margin_per_patient_day,operating_margin
flag_negative_income,,,,,
Profitable,0.94,"9,600.86","9,801.31","1,168.39",0.13
Loss making,0.93,"8,433.39","6,793.47","-1,293.02",-0.20


Loss making hospitals differ most strongly on revenue and margin metrics rather than utilization.
Utilization is high for both groups, suggesting that many losses are not driven by empty beds alone.
Loss making hospitals generate substantially less revenue per patient day and operate with negative margins, indicating pricing, payer mix, or service mix issues in addition to cost control.

Classify loss making hospitals into a small number of failure modes using system wide benchmarks.
This enables prioritization based on how hospitals deviate from normal system performance.

In [28]:
# Compute system medians for driver thresholds
system_medians = df_metrics_base[metric_cols].median()

system_medians


utilization_rate              0.96
cost_per_patient_day      7,001.96
revenue_per_patient_day   6,548.92
margin_per_patient_day      184.41
operating_margin              0.04
dtype: float64

Assign each hospital to a single primary loss driver based on how it deviates from system median performance.
Each hospital gets one dominant explanation to keep actions clear.

Driver logic
Profitable
Net income is non negative

High cost structure
Loss making and cost per patient day above system median

Low utilization
Loss making and utilization below system median

If a hospital meets both conditions, high cost takes priority because it dominates dollar impact.

In [29]:
# Start with a copy for driver classification
df_drivers = df_metrics_base.copy()

df_drivers["loss_driver"] = "Profitable"

# Identify loss making hospitals
loss_mask = df_drivers["flag_negative_income"]

# High cost structure takes priority
high_cost_mask = (
    loss_mask &
    (df_drivers["cost_per_patient_day"] > system_medians["cost_per_patient_day"])
)

df_drivers.loc[high_cost_mask, "loss_driver"] = "High cost structure"

# Low utilization for remaining loss makers
low_util_mask = (
    loss_mask &
    (df_drivers["loss_driver"] == "Profitable") &
    (df_drivers["utilization_rate"] < system_medians["utilization_rate"])
)

df_drivers.loc[low_util_mask, "loss_driver"] = "Low utilization"

# Sanity check counts
df_drivers["loss_driver"].value_counts()


loss_driver
Profitable             341
High cost structure     57
Low utilization         37
Name: count, dtype: int64

Quantify total financial impact by loss driver to identify where intervention produces the highest return.

In [30]:
# Aggregate total net income by loss driver
driver_financials = (
    df_drivers
    .groupby("loss_driver")["net_income"]
    .agg(
        hospital_count="count",
        total_net_income="sum",
        average_net_income="mean"
    )
    .sort_values("total_net_income")
)

driver_financials


,hospital_count,total_net_income,average_net_income
loss_driver,,,
High cost structure,57,-2101586205,"-36,869,933.42"
Low utilization,37,-322212445,"-8,708,444.46"
Profitable,341,16373198301,"48,015,244.28"


High cost structure hospitals represent the largest source of losses by a wide margin, accounting for over two billion in negative net income.
Low utilization hospitals contribute losses as well, but their total financial impact is materially smaller.
Most system profit is generated by a large group of profitable hospitals, masking severe losses in a smaller subset.

Identify structural characteristics where losses concentrate to guide targeted policy and operational strategies.
This helps leadership focus interventions on hospital types with repeatable risk patterns.

In [31]:
# Loss concentration by bed size category
bed_size_summary = (
    df_drivers
    .groupby("bed_size_category")["net_income"]
    .agg(
        hospital_count="count",
        loss_hospital_count=lambda x: (x < 0).sum(),
        total_net_income="sum",
        average_net_income="mean"
    )
    .sort_values("total_net_income")
)

bed_size_summary


,hospital_count,loss_hospital_count,total_net_income,average_net_income
bed_size_category,,,,
1 - 49,78,17,215297027,"2,760,218.29"
100 - 149,63,28,329363164,"5,227,986.73"
50 - 99,70,29,357979017,"5,113,985.96"
150 - 199,48,14,942361972,"19,632,541.08"
200 - 299,62,17,1024710929,"16,527,595.63"
300 - 499,77,26,4559920182,"59,219,742.62"
500 +,37,5,6519767360,"176,209,928.65"


Losses are not concentrated in the largest hospitals.
Small and mid sized hospitals have a higher proportion of loss making facilities, but large hospitals generate the majority of system profit.
Mid sized hospitals, particularly in the 100 to 299 bed range, represent a meaningful risk area due to higher loss frequency combined with limited profit offset.

This suggests that structural efficiency challenges are more common in smaller facilities, even if absolute dollars are larger in big systems.

In [32]:
# Loss concentration by teaching rural status
teaching_summary = (
    df_drivers
    .groupby("teaching_rural")["net_income"]
    .agg(
        hospital_count="count",
        loss_hospital_count=lambda x: (x < 0).sum(),
        total_net_income="sum",
        average_net_income="mean"
    )
    .sort_values("total_net_income")
)

teaching_summary


,hospital_count,loss_hospital_count,total_net_income,average_net_income
teaching_rural,,,,
Rural,59,13,429837928,"7,285,388.61"
Teaching,41,8,6505914760,"158,680,847.80"
Non-Teaching,335,115,7013646963,"20,936,259.59"


### Key findings

High cost structure is the dominant driver of hospital losses, contributing over two billion in negative net income.
Low utilization contributes to losses but is a secondary driver relative to cost intensity.

Losses are more frequent among small and mid sized hospitals, even though large hospitals generate most system profit.
This indicates risk concentration rather than absolute dollar dominance in smaller facilities.

Non teaching hospitals account for the majority of total losses due to volume, while teaching hospitals are strongly profitable on average.
Rural hospitals show moderate profit overall but still include a meaningful subset of loss makers.

Decision implications

Interventions should prioritize cost structure review before utilization expansion.
Small and mid sized non teaching hospitals represent the highest risk density for sustained losses.
Monitoring should focus on cost per patient day, revenue per patient day, and operating margin rather than utilization alone.

In [33]:
df_drivers.to_csv("df_drivers.csv", index=False)
